### Problem Statement

Use Case: Developing an LLM chatbot that would give product details

Requirements: 
*  Name of the product. 
*  Description of the product. 
*  Price of the product in USD

In [17]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate
import pprint

In [3]:
## Langchain and Langsmith tracing
os.environ['LANGCHAIN_API_KEY'] = os.getenv('LANGCHAIN_API_KEY')
os.environ['LANGCHAIN_PROJECT'] = os.getenv('LANGCHAIN_PROJECT')
os.environ["LANGCHAIN_TRACING_V2"]="true"

## Getting Froq API key
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [ ]:
llm = ChatGroq(model='gemma2-9b-it')
result = llm.invoke('Give me the details of the product iphone 13. The product description should not exceed 150 words. Also get the price of the product in USD, make sure the price is rounded off to 2 decimal place. Give the product details in the following format: Return a JSON object.')
print(result.content)

```json
{
  "name": "iPhone 13",
  "description": "The iPhone 13 boasts a stunning Super Retina XDR display, a powerful A15 Bionic chip, and a dual-camera system that captures incredible photos and videos. With an all-day battery life, advanced safety features like Crash Detection and Emergency SOS via satellite, and a sleek, durable design, the iPhone 13 is a premium smartphone experience.",
  "price": 599.00
}
``` 


Please note: The price listed is for the base model (128GB) and may vary depending on storage capacity, carrier, and retailer.



In [21]:
llm_openai = ChatOpenAI(model = "o1-mini")
result = llm_openai.invoke('Give me the details of the product iphone 13. The product description should not exceed 150 words. Also get the price of the product in USD, make sure the price is rounded off to 2 decimal place. Give the product details in the following format: Return a JSON object.')
print(result.content)

```json
{
  "product": "iPhone 13",
  "description": "The iPhone 13 showcases a sleek design with a durable aerospace-grade aluminum frame and Ceramic Shield front cover for enhanced durability. Powered by the A15 Bionic chip, it delivers exceptional performance and efficiency. The dual-camera system features advanced capabilities like Night mode, Deep Fusion, and Smart HDR 4, enabling users to capture stunning photos and videos in any lighting condition. The Super Retina XDR display offers vibrant colors and sharp details, perfect for multimedia experiences. With 5G connectivity, enjoy faster download and streaming speeds. Additional features include improved battery life, enhanced Face ID for security, and support for MagSafe accessories, making the iPhone 13 a versatile and powerful smartphone choice for everyday use.",
  "price_usd": 599.00
}
```


In [ ]:
## Creating Product Pydantic Class
class Product_Details(BaseModel):
    name : str
    description : str
    price : float=Field(description="The price is in USD")

In [ ]:
## Create the output parser
parser = JsonOutputParser(pydantic_object=Product_Details)


In [ ]:
## Creating the prompts
prompt = PromptTemplate(
    template="Give me the details of the product {product_name}. The product description should not exceed 150 words. Also get the price of the product in USD, make sure the price is rounded off to 2 decimal place. Give the product details in {format_instruction}",
    input_variables=["product_name"],
    partial_variables={"format_instruction":parser.get_format_instructions()}
)

input_variables=['product_name'] input_types={} partial_variables={'format_instruction': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "type": "string"}, "description": {"title": "Description", "type": "string"}, "price": {"description": "The price is in USD", "title": "Price", "type": "number"}}, "required": ["name", "description", "price"]}\n```'} template='Give me the details of the product {product_name}. The product description should not exceed 150 words. Also get the price of the product in USD, make sure the price is rounded off to

In [11]:
chain = prompt | llm | parser

In [ ]:
response = chain.invoke({"product_name":"iphone 13"})
pprint.pprint(response)

{'description': 'The iPhone 13 boasts a Super Retina XDR display,  an A15 '
                'Bionic chip for blazing-fast performance, and a powerful '
                'dual-camera system for stunning photos and videos. It also '
                'features an advanced battery life and is available in a range '
                'of vibrant colors.',
 'name': 'iPhone 13',
 'price': 699.0}


In [16]:
chain2 = prompt | llm_openai | parser

In [18]:
response = chain2.invoke({"product_name":"iphone 13"})
pprint.pprint(response)

{'description': 'The iPhone 13 features a sleek design with a Ceramic Shield '
                'front cover and a 6.1-inch Super Retina XDR display. Powered '
                'by the A15 Bionic chip, it delivers exceptional performance '
                'and efficiency. The dual-camera system includes advanced '
                'capabilities like Night mode, Deep Fusion, and Smart HDR 4, '
                'enabling stunning photos and videos in any condition. With 5G '
                'connectivity, users experience faster download and streaming '
                'speeds. The device offers improved battery life, running on '
                'the latest iOS for a seamless user experience. Available in '
                'multiple colors and storage options, the iPhone 13 combines '
                'style, functionality, and cutting-edge technology.',
 'name': 'iPhone 13',
 'price': 799.0}
